In [44]:
!pip install pdfplumber
!pip install torch
!pip install transformer
!pip install openai-embeddings
!pip install vector transformer

ERROR: Could not find a version that satisfies the requirement transformer (from versions: none)
ERROR: No matching distribution found for transformer
ERROR: Could not find a version that satisfies the requirement openai-embeddings (from versions: none)
ERROR: No matching distribution found for openai-embeddings
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.3/171.3 kB 3.6 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement transformer (from versions: none)
ERROR: No matching distribution found for transformer


In [9]:
import requests
import pdfplumber
import transformers
from transformers import pipeline, AutoModelForQuestionAnswering
import torch
# Download the PDF file from the URL
pdf_url = 'https://pgcag.files.wordpress.com/2010/01/48lawsofpower.pdf'
response = requests.get(pdf_url)
with open("downloaded_file.pdf", "wb") as pdf_file:
    pdf_file.write(response.content)

# Read data from the PDF
with pdfplumber.open("downloaded_file.pdf") as pdf:
    text = ""
    for page in pdf.pages:
        text += page.extract_text()


In [34]:
# prompt: code for Split the text into chunks for better processing

from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# Split the text into chunks of 100024 tokens
chunks = tokenizer.batch_encode_plus([text], return_tensors="pt", max_length=1024, truncation=True)["input_ids"]

# Convert the chunks to tensors

chunks=tuple(chunks)
chunks = torch.stack(chunks)
print(chunks)




tensor([[ 101,  160, 6258,  ..., 2844, 1103,  102]])


In [28]:

import transformers
model_name = "bert-base-uncased"
model = AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased")
tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-uncased")


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

In [49]:
import torch.nn as nn
qa_outputs_bias = torch.nn.Parameter(torch.zeros(model.qa_outputs.bias.shape))
qa_outputs_weight = torch.nn.Parameter(torch.zeros(model.qa_outputs.weight.shape))

model.qa_outputs.bias = qa_outputs_bias
model.qa_outputs.weight = qa_outputs_weight

model.save_pretrained("model_with_added_weights")

# Ask a question about the PDF content
question = input("Ask a question about the PDF content: ")

# Get the answer with text generation
answer = qa_pipeline(question=question, context=text)

# Print the answer
print(f"Question: {question}")
print(f"Answer: {answer['answer']}")

Ask a question about the PDF content: MASTER THE ART OF TIMING
Question: MASTER THE ART OF TIMING
Answer: slander
